In [42]:
import requests
from bs4 import BeautifulSoup
from csv import reader, writer

In [43]:
url = "https://www.rithmschool.com/blog"
response = requests.get(url)

In [44]:
soup = BeautifulSoup(response.text, "html.parser")

In [45]:
articles = soup.select("article")

In [47]:
with open("blog_data", "w") as csv_file:
    csv_writer = writer(csv_file)
    csv_writer.writerow(['Title', 'Link', 'Date'])
    
    for article in articles:
        a_tag = article.find("a")

        title = a_tag.get_text()
        url = a_tag["href"]
        date = article.find("time")['datetime']
        csv_writer.writerow([title, url, date])